# Selenium 구글 이미지 스크래핑

## Selenium 및 웹 드라이버 설치

In [14]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver /usr/bin')

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromi

In [15]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')



## 필요 라이브러리 import

In [16]:
import os
import time
import socket

from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image


## `scroll_down()`: 스크롤을 내리는 함수

In [44]:
def scroll_down():
  scroll_count = 0
  print('[scroll_down(): 스크롤 다운 시작]')

  last_height = wd.execute_script('return document.body.scrollHeight')
  after_click = False

  while True:
    print(f"[스크롤 다운: {scroll_count}]")
    wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    scroll_count += 1
    time.sleep(1)

    new_height = wd.execute_script('return document.body.scrollHeight')

    if last_height == new_height:
      if after_click is True:
        break
      else:
        try:
          more_button = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input')
          if more_button.is_displayed():
            more_button.click()
            after_click = True
        except NoSuchElementException as e:
          print(e)
          break

    last_height = new_height

## `click_and_save()`: 썸네일 이미지 선택 후 원본 이미지 저장

In [53]:
def click_and_save(dir_name, index, img, img_list_length):
  global scraped_count

  try:
    img.click()
    wd.implicitly_wait(3)
    src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img').get_attribute('src')
    if src.split('.')[-1] == 'png':
      urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + '.png')
      print(f"{index+1}/{img_list_length} PNG 이미지 저장")
    else:
      urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + '.jpg')
      print(f"{index+1}/{img_list_length} JPG 이미지 저장")

    scraped_count += 1

  except HTTPError as e:
    print(e)
    pass
  except ElementClickInterceptedException as e:
    print(e)
    pass

## `scraping()`: 구글 이미지 스크래핑 시작

In [55]:
def scraping(dir_name, query):
  global scraped_count

  url = f'https://www.google.com/search?q={query}&sxsrf=ALeKk00EqF41mymgTFoQlJsCeLou2H3zdw:1597211939222&source=lnms&tbm=isch&sa=X&ved=2ahUKEwinqpKS_pTrAhVX62EKHWPSDx8Q_AUoAXoECAwQAw&biw=869&bih=964'

  wd.get(url)
  wd.maximize_window()

  scroll_down()

  div = wd.find_element_by_xpath('//*[@id="islrg"]/div[1]')
  img_list = div.find_elements_by_css_selector('div.bRMDJf.islir > img')
  print(img_list)

  for index, img in enumerate(img_list):
    try:
      click_and_save(dir_name, index, img, len(img_list))
    except ElementClickInterceptedException as e:
      print(e)
      wd.execute_script('window.scrollTo(0, window.scrollY + 100')
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except NoSuchElementException as e:
      print(e)
      wd.execute_script('window.scrollTo(0, window.scrollY + 100')
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except ConnectionResetError as e:
      print(e)
      pass
    except URLError as e:
      print(e)
      pass
    except socket.timeout as e:
      print(e)
      pass
    except socket.gaierror as e:
      print(e)
      pass
    except ElementNotInteractableException as e:
      print(e)
      break

  try:
    print('[스크래핑 종료 (성공률: %.2f%%)]' % (scraped_count / len(img_list) * 100.0))
  except ZeroDivisionError as e:
    print(e)
  
  wd.quit()

## `filter_and_remove()`: 일정 해상도 이하이거나 손상된 이미지 제거

In [30]:
def filter_and_remove(dir_name,query, filter_size):
  filtered_count = 0

  for index, file_name in enumerate(os.listdir(dir_name)):
    try:
      file_path = os.path.join(dir_name, file_name)
      img = Image.open(file_path)

      if img.width < filter_size and img.height < filter_size:
        img.close()
        os.remove(file_path)
        print(f'{index} 이미지 제거')
        filtered_count += 1
    except OSError as e:
      print(e)
      os.remove(file_path)
      filtered_count += 1

  print(f'[이미지 제거 개수: {filtered_count}/{scraped_count}]')

## 이미지 확인

In [57]:
socket.setdefaulttimeout(30)

wd = webdriver.Chrome('chromedriver',options=chrome_options)
scraped_count = 0

path = "./"
query = input(' 검색어 입력 : ')

dir_name = path + query
try:
  os.makedirs(dir_name)
except:
  pass

print(f'[{dir_name} 디렉토리 생성]')

scraping(dir_name, query)
filter_and_remove(dir_name,query, 400)

 검색어 입력 : sea
[./sea 디렉토리 생성]
[scroll_down(): 스크롤 다운 시작]
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]
[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
[스크롤 다운: 8]
[스크롤 다운: 9]
[스크롤 다운: 10]
[스크롤 다운: 11]
[<selenium.webdriver.remote.webelement.WebElement (session="0c03facce61d2fc816b423a672103141", element="40fcdaab-b476-4260-92da-b1a212708c16")>, <selenium.webdriver.remote.webelement.WebElement (session="0c03facce61d2fc816b423a672103141", element="4947b88b-f733-4818-adca-b8e47cffbbe9")>, <selenium.webdriver.remote.webelement.WebElement (session="0c03facce61d2fc816b423a672103141", element="cf25328c-49ba-4a48-bcdd-9e84c7c527e2")>, <selenium.webdriver.remote.webelement.WebElement (session="0c03facce61d2fc816b423a672103141", element="608dea48-372f-4b6d-9867-a7d8ea6bd663")>, <selenium.webdriver.remote.webelement.WebElement (session="0c03facce61d2fc816b423a672103141", element="9408b418-6487-4b4f-bd09-6c0ab3b0e632")>, <selenium.webdriver.remote.webelement.WebElement (session="0c03facce61d2fc81

In [59]:
!ls sea

100.jpg  18.jpg   272.jpg  348.jpg  420.jpg  489.jpg  563.jpg  635.jpg	69.jpg
102.jpg  191.jpg  274.jpg  349.jpg  421.jpg  491.jpg  564.jpg  637.jpg	6.jpg
105.jpg  192.jpg  275.jpg  34.jpg   422.jpg  492.jpg  566.jpg  638.jpg	701.jpg
107.jpg  193.jpg  276.png  351.jpg  424.jpg  494.jpg  568.jpg  639.jpg	705.jpg
115.jpg  195.jpg  277.jpg  352.jpg  426.jpg  497.jpg  569.jpg  640.jpg	708.jpg
116.jpg  196.jpg  281.jpg  353.jpg  428.jpg  498.jpg  571.jpg  643.jpg	710.jpg
120.jpg  197.jpg  282.jpg  355.jpg  429.jpg  499.jpg  572.jpg  647.jpg	712.jpg
124.jpg  19.jpg   283.jpg  358.jpg  42.jpg   502.jpg  575.jpg  648.jpg	714.png
126.jpg  200.jpg  284.jpg  359.jpg  430.jpg  503.jpg  576.jpg  649.jpg	715.jpg
127.jpg  201.jpg  287.jpg  364.jpg  431.jpg  504.jpg  581.jpg  651.png	71.jpg
128.jpg  205.jpg  290.jpg  365.jpg  435.jpg  505.jpg  582.jpg  652.jpg	721.jpg
129.jpg  206.jpg  291.jpg  367.jpg  436.jpg  508.png  584.jpg  653.jpg	722.jpg
12.jpg	 207.jpg  294.jpg  369.jpg  438.jpg  512.jpg  585

In [56]:
!rmdir sea

In [60]:
from google.colab import files
files.download('./sea/18.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>